In [1]:
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import pickle
import requests
import re

In [2]:
team_abbrevs = ['crd', 'atl', 'rav', 'buf', 'car', 'chi', 'cin', 'cle', 'dal', 'den',
                'det', 'gnb', 'htx', 'clt', 'jax', 'kan', 'mia', 'min', 'nwe', 'nor',
                'nyg', 'nyj', 'rai', 'phi', 'pit', 'sdg', 'sfo', 'sea', 'ram', 'tam',
                'oti', 'was']

In [3]:
url = "https://www.pro-football-reference.com/teams/chi/2020.htm"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

In [4]:
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" data-root="/home/pfr/build" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en">
 <head>
  <!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
  <script async="true" type="text/javascript">
   (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://quantcast.mgr.consensu.org'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, '/choice.js')
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    var TCF_LOCATOR_NAME = '__tcfapiLocator';
	    var queue = [];
	    var win = window;
	    var cmpFrame;
	    
	    function addFrame() {
		var doc = win.document;
		var otherCMP = !!(win.frames[TCF_LOCATOR_NAME]);
		
		if (!otherCMP

In [5]:
len(soup.find_all('tbody'))

3

In [10]:
tables = soup.find_all('tbody')
sched_table = tables[1].find_all('th')
for row in sched_table:
    print(row.prettify())

<th class="right" csk="1" data-stat="week_num" scope="row">
 1
</th>

<th class="right" csk="2" data-stat="week_num" scope="row">
 2
</th>

<th class="right" csk="3" data-stat="week_num" scope="row">
 3
</th>

<th class="right" csk="4" data-stat="week_num" scope="row">
 4
</th>

<th class="right" csk="5" data-stat="week_num" scope="row">
 5
</th>

<th class="right" csk="6" data-stat="week_num" scope="row">
 6
</th>

<th class="right" csk="7" data-stat="week_num" scope="row">
 7
</th>

<th class="right" csk="8" data-stat="week_num" scope="row">
 8
</th>

<th class="right" csk="9" data-stat="week_num" scope="row">
 9
</th>

<th class="right" csk="10" data-stat="week_num" scope="row">
 10
</th>

<th class="right" csk="10.8" data-stat="week_num" scope="row">
 11
</th>

<th class="right" csk="12" data-stat="week_num" scope="row">
 12
</th>

<th class="right" csk="13" data-stat="week_num" scope="row">
 13
</th>

<th class="right" csk="14" data-stat="week_num" scope="row">
 14
</th>

<th clas

In [21]:
tables = soup.find_all('tbody')
sched_table = tables[1].find_all('tr')

for row in sched_table:
    print(row)

<tr><th class="right" csk="1" data-stat="week_num" scope="row">1</th><td class="left" data-stat="game_day_of_week">Sun</td><td class="left" csk="2020-09-13" data-stat="game_date">September 13</td><td class="right" csk="13.00" data-stat="game_time">1:00PM ET</td><td class="center" data-stat="boxscore_word"><a href="/boxscores/202009130det.htm">boxscore</a></td><td class="right" data-stat="game_outcome">W</td><td class="left iz" data-stat="overtime"></td><td class="center" data-stat="team_record">1-0</td><td class="right" data-stat="game_location">@</td><td class="left" data-stat="opp"><a href="/teams/det/2020.htm">Detroit Lions</a></td><td class="right" data-stat="pts_off">27</td><td class="right" data-stat="pts_def">23</td><td class="right" data-stat="first_down_off">26</td><td class="right" data-stat="yards_off">363</td><td class="right" data-stat="pass_yds_off">214</td><td class="right" data-stat="rush_yds_off">149</td><td class="right iz" data-stat="to_off"></td><td class="right" da

In [26]:
games = []

for row in sched_table:
    game = {}
    
    try: 
        opponent = re.search(r"href=\"/teams/([\w]{3}).*", str(row))[1]
        game['opp'] = opponent
    except:
        pass
    try:
        week_num = re.search(r"\"week_num\" scope=\"row\">([0-9]+)", str(row))[1]
        game['week_num'] = week_num
    except:
        pass
        
#     print(opponent)
    results = re.findall(r"data-stat=\"([\w]+)\">([@\w-]+)", str(row))

    for result in results:
#         print(result)
        if result != []:
            stat, val = result
            try:
                game[stat] = int(val)
            except:
                game[stat] = val
    if len(game) > 0:
        games.append(game)
    else:
        continue

In [27]:
games

[{'opp': 'det',
  'week_num': '1',
  'game_day_of_week': 'Sun',
  'game_date': 'September',
  'game_time': 1,
  'game_outcome': 'W',
  'team_record': '1-0',
  'game_location': '@',
  'pts_off': 27,
  'pts_def': 23,
  'first_down_off': 26,
  'yards_off': 363,
  'pass_yds_off': 214,
  'rush_yds_off': 149,
  'first_down_def': 25,
  'yards_def': 426,
  'pass_yds_def': 288,
  'rush_yds_def': 138,
  'to_def': 1,
  'exp_pts_off': 10,
  'exp_pts_def': -7,
  'exp_pts_st': 0},
 {'opp': 'nyg',
  'week_num': '2',
  'game_day_of_week': 'Sun',
  'game_date': 'September',
  'game_time': 1,
  'game_outcome': 'W',
  'team_record': '2-0',
  'pts_off': 17,
  'pts_def': 13,
  'first_down_off': 18,
  'yards_off': 304,
  'pass_yds_off': 169,
  'rush_yds_off': 135,
  'to_off': 2,
  'first_down_def': 22,
  'yards_def': 295,
  'pass_yds_def': 220,
  'rush_yds_def': 75,
  'to_def': 2,
  'exp_pts_off': 3,
  'exp_pts_def': 0,
  'exp_pts_st': -3},
 {'opp': 'atl',
  'week_num': '3',
  'game_day_of_week': 'Sun',
  '

In [8]:
games

[{'opp': 'det',
  'game_day_of_week': 'Sun',
  'game_date': 'September',
  'game_time': 1,
  'game_outcome': 'W',
  'team_record': '1-0',
  'game_location': '@',
  'pts_off': 27,
  'pts_def': 23,
  'first_down_off': 26,
  'yards_off': 363,
  'pass_yds_off': 214,
  'rush_yds_off': 149,
  'first_down_def': 25,
  'yards_def': 426,
  'pass_yds_def': 288,
  'rush_yds_def': 138,
  'to_def': 1,
  'exp_pts_off': 10,
  'exp_pts_def': -7,
  'exp_pts_st': 0},
 {'opp': 'nyg',
  'game_day_of_week': 'Sun',
  'game_date': 'September',
  'game_time': 1,
  'game_outcome': 'W',
  'team_record': '2-0',
  'pts_off': 17,
  'pts_def': 13,
  'first_down_off': 18,
  'yards_off': 304,
  'pass_yds_off': 169,
  'rush_yds_off': 135,
  'to_off': 2,
  'first_down_def': 22,
  'yards_def': 295,
  'pass_yds_def': 220,
  'rush_yds_def': 75,
  'to_def': 2,
  'exp_pts_off': 3,
  'exp_pts_def': 0,
  'exp_pts_st': -3},
 {'opp': 'atl',
  'game_day_of_week': 'Sun',
  'game_date': 'September',
  'game_time': 1,
  'game_outcom

In [39]:
games = []
start_year = 2019
end_year = 2020

for team in team_abbrevs:
    
    for year in range(start_year, end_year + 1):
        
        url = f"https://www.pro-football-reference.com/teams/{team}/{year}.htm"

        response = requests.get(url)
        if response.status_code == 404:
            continue

        soup = BeautifulSoup(response.text, "lxml")
        tables = soup.find_all('tbody')
        sched_table = tables[1].find_all('tr')
        
        for row in sched_table:
            game = {}
            game["team"] = team
            game["year"] = year

            try: 
                opponent = re.search(r"href=\"/teams/([\w]{3}).*", str(row))[1]
                game['opp'] = opponent
            except:
                pass
            
            # get week num (syntax is a little different)
            try:
                week_num = re.search(r"\"week_num\" scope=\"row\">([\w ]+)", str(row))[1]
                game['week_num'] = week_num
                
            except:
                pass
            
        #     print(opponent)
            results = re.findall(r"data-stat=\"([\w]+)\">([\w-]+)", str(row))
            for result in results:
        #         print(result)
                if result != []:
                    stat, val = result
                    try:
                        game[stat] = float(val)
                    except:
                        game[stat] = val
            if len(game) > 0:
                games.append(game)
            else:
                continue
            
        

KeyboardInterrupt: 

In [ ]:
game_df = pd.DataFrame(games)

In [ ]:
game_df.sample(20)

In [38]:
2 * 3**8

13122